**# User Input **
1. User will select a number for either Choose a file or Insert text as instructed.
2. User will select language type they want the output summary to be. [N>B. there is 8 type of language, user selected language will only be translate the summary]

In [1]:
#section for user input
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact

#For text
body = []
options = ['Choose a file', 'Insert text']

# Display the options to the user
print("Select an option:")
for i, option in enumerate(options, 1):
    print(f"{i}. {option}")

#user's choice
choice_index = input("Enter the number of your choice: ")

try:
    choice_index = int(choice_index)
    selected_option = options[choice_index - 1]

    if selected_option == 'Choose a file':
        file_name = input("Enter the file name: ")
        #file_name = 'AI.txt'
        try:
            with open(file_name, 'r') as file:
                body.extend(file.readlines())
        except FileNotFoundError:
            print("File not found. Please make sure the file exists.")
    elif selected_option == 'Insert text':
        user_input = input("Enter text for Summarization: ")
        body.extend(user_input.split('\n'))
    else:
        print("Invalid choice.")
except ValueError:
    print("Invalid input. Please enter a number.")

# for language datapath
data_paths = ["bangla.txt", "france.txt", "german.txt", "hindi.txt", "italy.txt", "japan.txt", "korea.txt", "turkey.txt"]
selected_path = data_paths[0]

def update_path(selected_path):
    index = data_paths.index(selected_path)
    global path
    path = selected_path

    print(f"Selected path: {selected_path}")
    print(f"Index number: {index}")
    print("\n")

path_dropdown = widgets.Dropdown(options=data_paths, value=data_paths[0], description='Select language:')
interact(update_path, selected_path=path_dropdown)
# content for summary
print("Content for Summarization:")
print(body)

Select an option:
1. Choose a file
2. Insert text
Enter the number of your choice: 1
Enter the file name: AI.txt


interactive(children=(Dropdown(description='Select language:', options=('bangla.txt', 'france.txt', 'german.tx…

Content for Summarization:
['What is artificial intelligence (AI)?\n', 'Artificial intelligence is the simulation of human intelligence processes by machines, especially computer systems. Specific applications of AI include expert systems, natural language processing, speech recognition and machine vision.\n', '\n', 'How does AI work?\n', 'As the hype around AI has accelerated, vendors have been scrambling to promote how their products and services use it. Often, what they refer to as AI is simply a component of the technology, such as machine learning. AI requires a foundation of specialized hardware and software for writing and training machine learning algorithms. No single programming language is synonymous with AI, but Python, R, Java, C++ and Julia have features popular with AI developers.\n', '\n', 'In general, AI systems work by ingesting large amounts of labeled training data, analyzing the data for correlations and patterns, and using these patterns to make predictions about 

**Pre-process**

In [2]:
text = ' '.join([str(elem) for elem in body])
text=text.replace("â\x80\x98","'")
text=text.replace("â\x80\x99","'")
text=text.replace("\n","")
text=text.replace("  ","")
text

'What is artificial intelligence (AI)? Artificial intelligence is the simulation of human intelligence processes by machines, especially computer systems. Specific applications of AI include expert systems, natural language processing, speech recognition and machine vision.How does AI work? As the hype around AI has accelerated, vendors have been scrambling to promote how their products and services use it. Often, what they refer to as AI is simply a component of the technology, such as machine learning. AI requires a foundation of specialized hardware and software for writing and training machine learning algorithms. No single programming language is synonymous with AI, but Python, R, Java, C++ and Julia have features popular with AI developers.In general, AI systems work by ingesting large amounts of labeled training data, analyzing the data for correlations and patterns, and using these patterns to make predictions about future states. In this way, a chatbot that is fed examples of 

In [3]:
print("Total number of letters :", len(text))
f=len(text.split())
print ("The number of words in the given text is : " +  str(f))

Total number of letters : 35857
The number of words in the given text is : 5433


**Import BERT summerizer**

Install bert-extractive-summarizer each time before execution

In [4]:
pip install bert-extractive-summarizer

Note: you may need to restart the kernel to use updated packages.


In [5]:
from summarizer import Summarizer,TransformerSummarizer

In [6]:
bert_model = Summarizer()
bert_summary = ''.join(bert_model(text, min_length=60))

F:\Users\hp\anaconda3\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


**Output Summary without Translation**

In [7]:
print(bert_summary)
summary = len(bert_summary.split())
print ("The number of words of the summary is : " +  str(summary))

Artificial intelligence is the simulation of human intelligence processes by machines, especially computer systems. Often, what they refer to as AI is simply a component of the technology, such as machine learning. No single programming language is synonymous with AI, but Python, R, Java, C++ and Julia have features popular with AI developers. New, rapidly improving generative AI techniques can create realistic text, images, music and other media. This aspect of AI programming is designed to continually fine-tune algorithms and ensure they provide the most accurate results possible. AI is important for its potential to change how we live, work and play. It has been effectively used in business to automate tasks done by humans, including customer service work, lead generation, fraud detection and quality control. In a number of areas, AI can perform tasks much better than humans. What are the advantages and disadvantages of artificial intelligence? AI is widely used in data-heavy indust

**Translation for selected Language**

In [8]:
import numpy as np
#import tensorflow as tf
from tensorflow import keras

In [10]:
print(path)
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

latent_dim = 256
batch_size = 64
num_samples = 5000

with open(path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

italy.txt
Number of samples: 5000
Number of unique input tokens: 67
Number of unique output tokens: 72
Max sequence length for inputs: 12
Max sequence length for outputs: 35


Define Encoder, Decoder and LSTM for Seq2Seq model

In [11]:
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [12]:
epochs = 50
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.2)
model.save("seq2seq_model")

Epoch 1/50
63/63 [==============================] - 79s 901ms/step - loss: 1.8237 - accuracy: 0.6235 - val_loss: 1.4670 - val_accuracy: 0.5943
Epoch 2/50
63/63 [==============================] - 48s 769ms/step - loss: 1.2425 - accuracy: 0.6480 - val_loss: 1.3165 - val_accuracy: 0.6440
Epoch 3/50
63/63 [==============================] - 43s 685ms/step - loss: 1.0892 - accuracy: 0.6907 - val_loss: 1.1875 - val_accuracy: 0.6732
Epoch 4/50
63/63 [==============================] - 51s 808ms/step - loss: 0.9826 - accuracy: 0.7227 - val_loss: 1.0905 - val_accuracy: 0.6827
Epoch 5/50
63/63 [==============================] - 56s 895ms/step - loss: 0.8976 - accuracy: 0.7423 - val_loss: 1.0284 - val_accuracy: 0.6974
Epoch 6/50
63/63 [==============================] - 52s 818ms/step - loss: 0.8368 - accuracy: 0.7561 - val_loss: 0.9792 - val_accuracy: 0.7026
Epoch 7/50
63/63 [==============================] - 56s 888ms/step - loss: 0.7930 - accuracy: 0.7643 - val_loss: 0.9499 - val_accuracy: 0.7075

INFO:tensorflow:Assets written to: seq2seq_model\assets


INFO:tensorflow:Assets written to: seq2seq_model\assets


MODEL ACCURACY

In [13]:
from sklearn.metrics import accuracy_score
predictions = model.predict([encoder_input_data, decoder_input_data])
predicted_labels = np.argmax(predictions, axis=-1)

true_labels = np.argmax(decoder_target_data, axis=-1)

accuracy = accuracy_score(true_labels.flatten(), predicted_labels.flatten())
accuracy = accuracy * 100
print(f"Final Accuracy: {accuracy}%")

157/157 [==============================] - 27s 136ms/step
Final Accuracy: 91.81428571428572%


In [14]:
model = keras.models.load_model("seq2seq_model")

encoder_inputs = model.input[0]
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0
        states_value = [h, c]
    return decoded_sentence

In [15]:
import nltk

nltk.download('punkt')

def process_text(sentence):
    print(f"Processing: {sentence}")

user_input = bert_summary
sentences = nltk.sent_tokenize(user_input)
translated_sentences = []

for sentence in sentences:
    process_text(sentence)

    input_seq = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype="float32")
    for t, char in enumerate(sentence):
        if t < max_encoder_seq_length and char in input_token_index:
            input_seq[0, t, input_token_index[char]] = 1.0

    if t + 1 < max_encoder_seq_length:
        input_seq[0, t + 1, input_token_index[" "]] = 1.0

    decoded_sentence = decode_sequence(input_seq)
    translated_sentences.append(decoded_sentence)

combined_translation = ''.join(translated_sentences)

print("-")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Processing: Artificial intelligence is the simulation of human intelligence processes by machines, especially computer systems.
1/1 [==============================] - 0s 63ms/step
Processing: Often, what they refer to as AI is simply a component of the technology, such as machine learning.
1/1 [==============================] - 0s 82ms/step
Processing: No single programming language is synonymous with AI, but Python, R, Java, C++ and Julia have features popular with AI developers.
1/1 [==============================] - 0s 47ms/step
Processing: New, rapidly improving generative AI techniques can create realistic text, images, music and other media.
1/1 [==============================] - 0s 62ms/step
Processing: This aspect of AI programming is designed to continually fine-tune algorithms and ensure they provide the most accurate results possible.
1/1 [==============================] - 0s 63ms/step
Processing: AI is important for its potential to change how we live, work and play.
1/1 [=

1/1 [==============================] - 0s 62ms/step
Processing: AI is widely used in data-heavy industries, including banking and securities, pharma and insurance, to reduce the time it takes to analyze big data sets.
1/1 [==============================] - 0s 63ms/step
Processing: Financial services, for example, routinely use AI to process loan applications and detect fraud.
1/1 [==============================] - 0s 47ms/step
Processing: An example here is the use of warehouse automation, which grew during the pandemic and is expected to increase with the integration of AI and machine learning.
1/1 [==============================] - 0s 62ms/step
Processing: The best AI translation tools deliver high levels of consistency, offering even small businesses the ability to reach customers in their native language.
1/1 [==============================] - 0s 63ms/step
Processing: AI can personalize content, messaging, ads, recommendations and websites to individual customers.
1/1 [============

1/1 [==============================] - 0s 63ms/step
Processing: Weak AI, also known as narrow AI, is designed and trained to complete a specific task.
1/1 [==============================] - 0s 62ms/step
Processing: Industrial robots and virtual personal assistants, such as Apple's Siri, use weak AI.
1/1 [==============================] - 0s 47ms/step
Processing: Strong AI, also known as artificial general intelligence (AGI), describes programming that can replicate the cognitive abilities of the human brain.
1/1 [==============================] - 0s 62ms/step
Processing: When presented with an unfamiliar task, a strong AI system can use fuzzy logic to apply knowledge from one domain to another and find a solution autonomously.
1/1 [==============================] - 0s 63ms/step
Processing: In theory, a strong AI program should be able to pass both a Turing test and the Chinese Room argument.
1/1 [==============================] - 0s 62ms/step
Processing: This is the science of getting 

1/1 [==============================] - 0s 63ms/step
Processing: Applications such as these collect personal data and provide financial advice.
1/1 [==============================] - 0s 63ms/step
Processing: The entertainment business uses AI techniques for targeted advertising, recommending content, distribution, detecting fraud, creating scripts and making movies.
1/1 [==============================] - 0s 63ms/step
Processing: Banks are successfully employing chatbots to make their customers aware of services and offerings and to handle transactions that don't require human intervention.
1/1 [==============================] - 0s 70ms/step
Processing: Banking organizations use AI to improve their decision-making for loans, set credit limits and identify investment opportunities.
1/1 [==============================] - 0s 63ms/step
Processing: Examples include automatically surfacing important information in business intelligence reports or highlighting important information in legal fil

1/1 [==============================] - 0s 58ms/step
Processing: A "Blueprint for an AI Bill of Rights" published in October 2022 by the White House Office of Science and Technology Policy (OSTP) guides businesses on how to implement ethical AI systems.
1/1 [==============================] - 0s 47ms/step
Processing: The Greek god Hephaestus was depicted in myths as forging robot-like servants out of gold.
1/1 [==============================] - 0s 63ms/step
Processing: And Warren McCulloch and Walter Pitts laid the foundation for neural networks.1950s.
1/1 [==============================] - 0s 47ms/step
Processing: One method for determining whether a computer has intelligence was devised by the British mathematician and World War II code-breaker Alan Turing.
1/1 [==============================] - 0s 62ms/step
Processing: In the mid-1960s, MIT Professor Joseph Weizenbaum developed ELIZA, an early NLP program that laid the foundation for today's chatbots.1970s and 1980s.
1/1 [============

1/1 [==============================] - 0s 66ms/step
Processing: IBM launched Watson and Google started its self-driving initiative, Waymo.2010s.
1/1 [==============================] - 0s 63ms/step
Processing: The abilities of language models such as ChatGPT-3, Google's Bard and Microsoft's Megatron-Turing NLG have wowed the world, but the technology is still in early stages, as evidenced by its tendency to hallucinate or skew answers.
1/1 [==============================] - 0s 94ms/step
Processing: This paved the way for the discovery of transformers that automate many aspects of training AI on unlabeled data.
1/1 [==============================] - 0s 78ms/step
Processing: The AI stack has also evolved rapidly over the last few years.
1/1 [==============================] - 0s 47ms/step
-


In [16]:
print("Translated List:")
print(translated_sentences)

Translated List:
['Co tiuda qui.\n', 'Chieniate a nussa.\n', 'Adesso ontrime.\n', 'Chi sono simminato.\n', 'È sto falicine.\n', 'Ti siuniate?\n', 'È caldo.\n', 'Sono no parui.\n', 'Chi è vigato?\n', 'Tuste inneriaci.\n', 'Porsa den  una vo.\n', 'Si sono salato?\n', 'Lui concongaio.\n', 'So puri le inato.\n', 'Io sono sura?\n', 'Di sorui.\n', 'Condite a na siana.\n', 'Abbiamo parlato.\n', 'Amnaitalo.\n', 'Ci diada a cusa.\n', 'Ci sino sonoscata.\n', 'Io lo conoglia.\n', 'È colta?\n', 'Mi vicci.\n', 'È strano.\n', "L'i corrigato.\n", 'Coctiluitemene Tom.\n', 'Lo ho sino cascota.\n', 'Pricate in  un pia.\n', 'Porta a cono.\n', 'Esamina quasta.\n', 'Chi sono sosattiato.\n', 'Mi striate.\n', 'Si marra di piete.\n', 'Stia bentenentene.\n', 'Tom sa mangiato?\n', 'Corri di anto.\n', 'Vessa prente.\n', 'Io lo incontrato.\n', "Lui co l' ccosto i na grigiono.\n", 'Masspo vine.\n', 'Lui cho mangato.\n', 'Ha galzato?\n', 'Laicho in letta?\n']


SHOW OUTPUT

Install flask-ngrok before each execution

In [17]:
!pip install flask-ngrok

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify

app = Flask(__name__)
run_with_ngrok(app)

@app.route('/get_output', methods=['GET'])
def get_output():
    output = {
        "Summerized sentences": sentences,
        "Translated List": translated_sentences,
        "Final Summerized translation": combined_translation
    }
    return jsonify(output)

app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://9c32-34-91-196-122.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


FINAL OUTPUT

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML

output_widget = widgets.Output(layout={'background-color': 'white', 'padding': '10px'})

def show_output(_):
    with output_widget:
        output_widget.clear_output(wait=True)

        display(HTML('<h3 style="background-color: Blue; padding: 10px;">Your Summarized Translated text is here:</h3>'))

        sentences_str = ''.join(sentences)
        #translated_sentences_str = '<br>'.join(translated_sentences)
        combined_translation_str = ''.join(combined_translation)

        display(HTML(f'<div style="background-color: #86608E; padding: 10px;"><b>Summerized sentences:</b><br>{sentences_str}</div>'))
        #display(HTML(f'<div style="background-color: #86608E; 10px;"><b>Translated List:</b><br><b>[</b>{translated_sentences_str}<b>]</b></div>'))
        display(HTML(f'<div style="background-color: #86608E; padding: 10px;"><b>Final Summerized translation:</b><br>{combined_translation_str}</div>'))

button = widgets.Button(description="Show Output")
button.on_click(show_output)

display(button)
display(output_widget)


Button(description='Show Output', style=ButtonStyle())

Output(layout=Layout(padding='10px'))